In [1]:
import pandas as pd
import math
from collections import Counter

In [3]:
data = pd.read_csv('titanic.csv')
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

# 处理缺失值
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
data['Fare'].fillna(data['Fare'].mean(), inplace=True)

# 转换分类特征
label_encoder = {}
for column in ['Sex', 'Embarked']:
   label_encoder[column] = {label:idx for idx, label in enumerate(data[column].unique())}
   data[column] = data[column].map(label_encoder[column])
   
x = data[features].values
y = data[target].values

In [10]:
# 计算信息熵
def entropy(data):
    label_cnts = Counter(data)
    total_cnt = len(data)
    return -sum(
        (count / total_cnt) * math.log2(count / total_cnt) for count in label_cnts
    )


# 划分数据集
def split_ds(data, labels, axis, value):
    ret_ds = []
    ret_label = []
    for row, label in zip(data, labels):
        if row[axis] == value:
            reduced_row = list(row[:axis]) + list(row[axis + 1 :])
            ret_ds.append(reduced_row)
            ret_label.append(label)
    return ret_ds, ret_label


# 选择最佳划分特征
def choose_best_features(data, lbls):
    num_features = len(data[0])
    base_entropy = entropy(lbls)
    best_info_gain = 0.0
    best_feature = -1

    for i in range(num_features):
        unique_vals = set(row[i] for row in data)
        new_entropy = sum(
            (len(subset) / len(lbls)) * entropy(subset_labels)
            for val in unique_vals
            for subset, subset_labels in [split_ds(data, lbls, i, val)]
        )

        info_gain = base_entropy - new_entropy
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = i
    return best_feature, best_info_gain


# 多数表决
def majority_cnt(class_list):
    return Counter(class_list).most_common(1)[0][0]


# 创建决策树
def create_tree(
    data, labels, feature_names, max_depth=None, min_info_gain=0.01, depth=0
):
    class_list = labels
    if class_list.count(class_list[0]) == len(class_list):
        return class_list[0]
    if len(data[0]) == 0:
        return majority_cnt(class_list)
    if max_depth is not None and depth >= max_depth:
        return majority_cnt(class_list)

    best_feature, best_info_gain = choose_best_features(data, labels)
    if best_info_gain < min_info_gain:
        return majority_cnt(class_list)

    best_feature_name = feature_names[best_feature]
    tree = {best_feature_name: {}}
    unique_vals = set(row[best_feature] for row in data)
    for value in unique_vals:
        sub_feature_names = (
            feature_names[:best_feature] + feature_names[best_feature + 1 :]
        )
        subset, subset_labels = split_ds(data, labels, best_feature, value)
        tree[best_feature_name][value] = create_tree(
            subset,
            subset_labels,
            sub_feature_names,
            max_depth,
            min_info_gain,
            depth + 1,
        )

    return tree


# 后剪枝
def post_prune(tree, validation_data, validation_labels, feature_names):
    if not isinstance(tree, dict):
        return tree

    root = next(iter(tree))
    subtrees = tree[root]
    feature_index = feature_names.index(root)
    for key in subtrees:
        if isinstance(subtrees[key], dict):
            subtrees[key] = post_prune(
                subtrees[key], validation_data, validation_labels, feature_names
            )

    # 计算未剪枝误差
    unprune_error = sum(
        predict(tree, feature_names, sample) != label
        for sample, label in zip(validation_data, validation_labels)
    )
    
    # 计算剪枝后误差
    majority_class = majority_cnt(validation_labels)
    pruned_err = sum(majority_class != label for label in validation_labels)
    
    if pruned_err <= unprune_error:
        return majority_class
    else:
        return tree